# Projeto - Análise de Risco em Cartões de Crédito
Projeto desenvolvido por Thiago Bulgarelli

## Contextualização

A análise de riscos em cartões de crédito é um campo fundamental dentro do setor financeiro, que busca garantir a segurança das transações e a proteção tanto dos consumidores quanto das instituições financeiras.

A crescente popularidade do uso de cartões de crédito como meio de pagamento tem gerado um ambiente complexo e desafiador, no qual a detecção precoce de possíveis ameaças é essencial para evitar prejuízos financeiros significativos.

Um dos principais pilares da análise de riscos em cartões de crédito é a avaliação do histórico de pagamento do titular do cartão. Esse aspecto inclui a análise de como o cliente tem gerenciado suas dívidas e honrado seus compromissos financeiros no passado. A análise do histórico de pagamento é frequentemente realizada por meio de sistemas de pontuação de crédito, como o FICO Score nos EUA, que avaliam o risco de inadimplência com base em vários fatores, incluindo histórico de pagamento, dívidas atuais, duração do histórico de crédito e outros.

Outro ponto crítico na análise de riscos em cartões de crédito é a detecção de atividades suspeitas e a prevenção de fraudes. Isso envolve o monitoramento constante de transações em busca de padrões incomuns que possam indicar uma fraude em andamento. Técnicas avançadas, como aprendizado de máquina e anáise de Big Data, são frequentemente utilizadas para identificar comportamentos fraudulentos e tomar medidas rápidas para proteger os consumidores e as instituições financeiras.

A análise de riscos em cartões de crédito é uma discplina dinâmica e em constante evolução, essencial para garantir a estabilidade e a segurança do sistema financeiro. Ela desempenha um papel crucial na concessão responsável de crédito e na prevenção de fraudes, contribuindo para a confiança dos consumidores e a saúde financeira das instituições.

## Estudo Técnico Aplicado

Utilizando dados públicos, disponibilizados na plataforma do Kaggle pelo link https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction, vamos analisar os dados com mais profundidade, identificando oportunidades e insights ao Cliente. Nosso objetivo final é criar um classificador, determinando quais clientes são "BOM PAGADORES" ou "MAU PAGADORES" e prever a Probabilidade de Inadimplência dos mesmos.

### Sobre o Dataset

Os datasets disponibilizados conta com o seguinte detalhamento:

Dataset: application_record.csv
- ID = Identificador do Cliente
- CODE_GENDER = Gênero do Cliente
- FLAG_OWN_CAR = Refere-se a propriedade de carro pelo cliente
- FLAG_OWN_REALTY = Refere-se a propriedade residencial pelo cliente
- CNT_CHILDREN = Número de Filhos do Cliente
- AMT_INCOME_TOTAL = Receita Anual do Cliente
- NAME_INCOME_TYPE = Origem da Receita do Cliente
- NAME_EDUCATION_TYPE = Grau de Formação do Cliente
- NAME_FAMILY_STATUS = Estado Civil do Cliente
- NAME_HOUSING_TYPE = Tipo de Moradia
- DAYS_BIRTH = Data do Aniversário relativa ao dia atual (hoje = 0)
- DAYS_EMPLOYED = Total de dias Desempregado (valor positivo) e Empregado (Valor Negativo)
- FLAG_MOBILE = Refere-se a propriedade de Celular do Cliente
- FLAG_WORK_PHONE = Refere-se a propriedade de telefone corporativo do Cliente
- FLAG_PHONE = Refere-se a propriedade de telefone fixo do cliente
- FLAG_EMAIL = Refere-se a propriedade de Email do Cliente
- OCCUPATION_TYPE = Tipo de Ocupação do Cliente
- CNT_FAM_MEMBERS = Quantidade de pessoas na família do Cliente

Dataset: credit_record.csv
- ID = Identificador do Cliente
- MONTHS_BALANCE = Medida relativa do mês devedor, sendo 0 o mês atual
- STATUS = Tempo de Atraso nos Pagamentos DPD (Days Past Due): 
    - 0 - 1-29 dias, 
    - 1 - 30-59 dias, 
    - 2 - 60-89 dias, 
    - 3 - 90-119 dias, 
    - 4 - 120-149 dias, 
    - 5 - dividas baixadas com mais de 150 dias, 
    - C - Quitado no mês, 
    - X - Nenhum empréstimo no mês


### Carregando Dados e Pacotes

In [340]:
# Pacotes de Trabalho
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly
import seaborn as sbn
import sklearn as skl
import warnings
warnings.filterwarnings("ignore")

In [341]:
# Importando os dados
client_data = pd.read_csv("./application_record.csv")
dpd_data = pd.read_csv("./credit_record.csv")

In [342]:
# Visualizando os dados dos Clientes
client_data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [343]:
# Visualizando os dados de DPD
dpd_data.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


### Limpeza e Transformação dos Dados de Trabalho

In [344]:
# Renomeando as Variáveis do dataset client_data
nomes_var = ["Id", "Genero", "Carro_proprio", "Residencia_Propria", "Qtde_Filhos", "Receita_Anual", "Origem_Receita", "Escolaridade", "Estado_Civil", "Tipo_Moradia", "Aniv_Rel", "Dias_Empregado", "Possui_Celular", "Possui_Tel_Corp", "Possui_Tel_Fixo", "Possui_Email", "Ocupacao", "Tam_Familia"]
client_data.columns = nomes_var
client_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Id                  438557 non-null  int64  
 1   Genero              438557 non-null  object 
 2   Carro_proprio       438557 non-null  object 
 3   Residencia_Propria  438557 non-null  object 
 4   Qtde_Filhos         438557 non-null  int64  
 5   Receita_Anual       438557 non-null  float64
 6   Origem_Receita      438557 non-null  object 
 7   Escolaridade        438557 non-null  object 
 8   Estado_Civil        438557 non-null  object 
 9   Tipo_Moradia        438557 non-null  object 
 10  Aniv_Rel            438557 non-null  int64  
 11  Dias_Empregado      438557 non-null  int64  
 12  Possui_Celular      438557 non-null  int64  
 13  Possui_Tel_Corp     438557 non-null  int64  
 14  Possui_Tel_Fixo     438557 non-null  int64  
 15  Possui_Email        438557 non-nul

In [345]:
# Verificando dados ausentes no dataset client_data
client_data.isna().sum() # a Variável ocupação possui muitos dados NaN, vamos filtrar e entender o motivo

Id                         0
Genero                     0
Carro_proprio              0
Residencia_Propria         0
Qtde_Filhos                0
Receita_Anual              0
Origem_Receita             0
Escolaridade               0
Estado_Civil               0
Tipo_Moradia               0
Aniv_Rel                   0
Dias_Empregado             0
Possui_Celular             0
Possui_Tel_Corp            0
Possui_Tel_Fixo            0
Possui_Email               0
Ocupacao              134203
Tam_Familia                0
dtype: int64

In [346]:
# Filtrando o dataset client_data por ocupação com NaN
client_data[(client_data["Ocupacao"].isna() == True) & (client_data["Dias_Empregado"] < 0)].head()

,Id,Genero,Carro_proprio,Residencia_Propria,Qtde_Filhos,Receita_Anual,Origem_Receita,Escolaridade,Estado_Civil,Tipo_Moradia,Aniv_Rel,Dias_Empregado,Possui_Celular,Possui_Tel_Corp,Possui_Tel_Fixo,Possui_Email,Ocupacao,Tam_Familia
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
24,5008834,F,N,Y,1,112500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-10968,-1620,1,0,0,0,NaN,2.0
25,5008835,F,N,Y,1,112500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-10968,-1620,1,0,0,0,NaN,2.0
26,6153712,F,N,Y,1,112500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-10968,-1620,1,0,0,0,NaN,2.0


Entendemos que para os clientes que possuem dias empregados e a origem da receita preenchidos, podemos considerar que a ocupação apenas não foi informada, sendo assim aplicaremos como "Não Informada".

Para os clientes que não estão trabalhando, aplicaremos como "Desempregados".

In [347]:
# Aplicando a categoria "Nao_Informado" a Ocupacao com dados faltantes (NaN) com dias_empregados negativos
cond = (client_data["Ocupacao"].isna() == True) & (client_data["Dias_Empregado"] < 0)
client_data.loc[cond, "Ocupacao"] = "Nao_Informado"

# Aplicando a categoria "Desempregado" a Ocupacao com dados faltantes (NaN) com dias_empregados positivos
cond = (client_data["Ocupacao"].isna() == True) & (client_data["Dias_Empregado"] > 0)
client_data.loc[cond, "Ocupacao"] = "Desempregado"

# Visualizando o resultado das categorias em Ocupacao
client_data["Ocupacao"].value_counts()


Laborers                 78240
Desempregado             75329
Nao_Informado            58874
Core staff               43007
Sales staff              41098
Managers                 35487
Drivers                  26090
High skill tech staff    17289
Accountants              15985
Medicine staff           13520
Cooking staff             8076
Security staff            7993
Cleaning staff            5845
Private service staff     3456
Low-skill Laborers        2140
Secretaries               2044
Waiters/barmen staff      1665
Realty agents             1041
HR staff                   774
IT staff                   604
Name: Ocupacao, dtype: int64

Vamos verificar ainda se temos Id's repetidos e caso positivo, excluiremos de nossa base de dados.

In [348]:
duplicated_rows = client_data.duplicated(subset="Id", keep=False)
print(f"Existem {duplicated_rows.sum()} linhas duplicadas que necessitam ser excluídas")

Existem 94 linhas duplicadas que necessitam ser excluídas


In [349]:
# Excluindo linhas duplicadas
client_data.drop_duplicates(subset="Id", keep=False, inplace=True)
client_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438463 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Id                  438463 non-null  int64  
 1   Genero              438463 non-null  object 
 2   Carro_proprio       438463 non-null  object 
 3   Residencia_Propria  438463 non-null  object 
 4   Qtde_Filhos         438463 non-null  int64  
 5   Receita_Anual       438463 non-null  float64
 6   Origem_Receita      438463 non-null  object 
 7   Escolaridade        438463 non-null  object 
 8   Estado_Civil        438463 non-null  object 
 9   Tipo_Moradia        438463 non-null  object 
 10  Aniv_Rel            438463 non-null  int64  
 11  Dias_Empregado      438463 non-null  int64  
 12  Possui_Celular      438463 non-null  int64  
 13  Possui_Tel_Corp     438463 non-null  int64  
 14  Possui_Tel_Fixo     438463 non-null  int64  
 15  Possui_Email        438463 non-nul

In [350]:
# Analisando o dataset dpd_data para dados ausentes
dpd_data.isna().sum()

ID                0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

In [351]:
# Ajustando o nome das colunas
colunas = dpd_data.columns.str.capitalize()
dpd_data.columns = colunas
dpd_data.nunique()


Id                45985
Months_balance       61
Status                8
dtype: int64

Não temos dados ausentes no dataset dpd_data, portanto podemos prosseguir com as análises dos dados e aplicação de técnicas de credit scoring.

### Análise de Dados e Aplicação da Técnica Vintage Analysis
 

Nosso primeiro passo é utilizar uma técnica de classificação de crédito para criar nossa variável resposta. Para tal, vamos utilizar um DPD (Days Past Due) de 30 dias em uma janela de 12 meses, identificada por ever30mob12. Nesta técnica, classificaremos como "MAU PAGADOR" todos os clientes que tiverem ao menos uma ocorrência de inadimplência maior que 30 dias no período de 12 meses a contar da data de hoje.

Vamos utilizar o dataset dpd_data para este trabaho e então unificaremos os dataset's para estudarmos cada variável e seus comportamentos.

In [352]:
# Vizualizando o período de exposição do dataset pdp_data
dpd_data["Months_balance"] = dpd_data["Months_balance"].astype("int")

dpd_data["Months_balance"].value_counts()


-1     34436
-2     34209
 0     33856
-3     33854
-4     33365
       ...  
-56     2109
-57     1649
-58     1255
-59      820
-60      415
Name: Months_balance, Length: 61, dtype: int64

In [353]:
# Função para Classificação das Ocorrências em uma nova Coluna chamada Classe
def classificador(classe):
    class_90_menos = ["0", "C", "X"]
    if classe in class_90_menos:
        return 1
    else:
        return 0
    
# Filtrando o Dataset para exposição de 12 meses e Aplicando o Classificador

dpd_data["Months_balance"] = dpd_data["Months_balance"].astype(int)
dpd_data = dpd_data[(dpd_data["Months_balance"] >= -12)]
dpd_data["Classe"] = dpd_data["Status"].apply(classificador)
dpd_data["Classe"].value_counts()

1    409708
0      5155
Name: Classe, dtype: int64

In [354]:
# Criando um novo dataset com a Classe por ID
dpd_por_id = dpd_data.groupby(['Id']).mean()


In [355]:
# Deletando coluna Months_Balance
dpd_por_id.drop("Months_balance", axis=1, inplace=True)

Temos que ajustar os dados de Classe, visto que qualquer número diferente de 1 deve ser considerado 0, dentro do nosso período de análise de 12 meses.

In [357]:
# Ajustando as Classes de Agrupamento
def ajuste_de_classe(x):
    if x != 1:
        return 0
    else:
        return 1
    
# Aplicando a Série
dpd_por_id["Classe"] = dpd_por_id["Classe"].map(ajuste_de_classe)
dpd_por_id["Classe"].value_counts()

1    36940
0     2339
Name: Classe, dtype: int64

In [358]:
# Unificando os Datasets em um único de trabalho
dataset = pd.merge(left=client_data, right=dpd_por_id, on="Id")
dataset.isna().sum()

Id                    0
Genero                0
Carro_proprio         0
Residencia_Propria    0
Qtde_Filhos           0
Receita_Anual         0
Origem_Receita        0
Escolaridade          0
Estado_Civil          0
Tipo_Moradia          0
Aniv_Rel              0
Dias_Empregado        0
Possui_Celular        0
Possui_Tel_Corp       0
Possui_Tel_Fixo       0
Possui_Email          0
Ocupacao              0
Tam_Familia           0
Classe                0
dtype: int64

In [359]:
# Verificando o Balanceamento do Dataset
dataset["Classe"].value_counts()

1    27918
0     1833
Name: Classe, dtype: int64

Temos então um dataset desbalanceado, o que é normal para o problema de negócio, visto que temos mais adimplentes que inadimplentes em uma instituição financeira. No nosso caso, temos 6.56% de maus pagadores e 93.43% de bons pagadores.

#### Estudo dos Dados e Identificação de Padrões de Comportamento dos Tomadores de Crédito